In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"../../../..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD/MultiAgentOptimization/julia_bots_ws/Project.toml`
Precompiling project...
  ✓ Parsers
  ✓ JSON
  ✓ Conda
  ✓ PyCall
  ✓ RobotOS
  5 dependencies successfully precompiled in 12 seconds (9 already precompiled)
  3 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [2]:
#!/usr/bin/env julia167
#!/usr/bin/python
# Change to 'julia' instead of 'julia167' if you don't have a symlink setup

using RobotOS
using iLQGameSolver
using SparseArrays
using LinearAlgebra
using StaticArrays


@rosimport geometry_msgs.msg: Twist, Pose
@rosimport nav_msgs.msg.Odometry
rostypegen()
using .geometry_msgs.msg: Twist, Pose
using .nav_msgs.msg: Odometry

┌ Info: Precompiling RobotOS [22415677-39a4-5241-a37a-00beabbbdae8]
└ @ Base loading.jl:1342


In [3]:
# Setup the problem

dt = 0.1                    # Step size [s]
tf = 60.0                    # Horizon [s]
N = Int(tf/dt)         # Number of steps (knot points)
println(N)

# Define cost matrices 
nx = 4
nu = 2
Nplayer = 2

Nu = nu * Nplayer
Nx = nx * Nplayer

Q1 = sparse(zeros(Nx,Nx))     # State cost for agent 1
Q1[1:nx,1:nx] .= 2.0*I(nx)
Qn1 = 5.0*Q1                    # Terminal cost for agent 1

Q2 = sparse(zeros(Nx,Nx))     # State cost for agent 2
Q2[nx+1:2*nx,nx+1:2*nx] .= 2.0*I(nx)
Qn2 = 5.0*Q2                    # Terminal cost for agent 2

# Q3 = sparse(zeros(Nx,Nx))     # State cost for agent 2
# Q3[2*nx+1:3*nx,2*nx+1:3*nx] .= 3.0*I(nx)
# Qn3 = Q3                    # Terminal cost for agent 2

R11 = sparse(5.0*I(nu))              # Control cost for player 1
R12 = sparse(0.0*I(nu))     # Control cost for player 1 associated with player 2's controls
# R13 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
R21 = sparse(0.0*I(nu))     # Control cost for player 2 associated with player 1's controls
R22 = sparse(5.0*I(nu))              # Contorl cost for player 2
# R23 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R31 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R32 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R33 = sparse(1.0*I(2))     # Control cost for player 2 associated with player 1's controls

dmax = 0.4                  # Distance that both agents should keep between each other [m]
ρ = 100.0                   # Penalty factor for violating the distance constraint

# Q's are stacked vertically
Q = sparse(zeros(Float32, Nx*Nplayer, Nx))
# @show size([Q1; Q2; Q3]), size(Q)
Q .= [Q1; Q2]
#Q .= [Q1; Q2; Q3]

# Qn's are stacked vertically
Qn = sparse(zeros(Float32, Nx*Nplayer, Nx))
Qn .= [Qn1; Qn2]
#Qn .= [Qn1; Qn2; Qn3]

# R's are stacked as a matrix
R = sparse(zeros(Float32, Nu, Nu))
R .= [R11 R12; R21 R22]
#R .= [R11 R12 R13; R21 R22 R23; R31 R32 R33]

NHor = 100
tol = 1e-2

game = iLQGameSolver.GameSetup(nx, nu, Nplayer, Q, R, Qn, dt, tf, NHor, dmax, ρ, tol)

solver = iLQGameSolver.iLQSetup(Nx, Nu, Nplayer, NHor);

600


In [4]:
function nominal_trajectory(x0, xgoal, N,dt)
    xref = [fill(NaN, length(x0)) for k = 1:N]
    xref[1] .= x0
    V1 = (xgoal[1:2]-x0[1:2])/((N-1)*dt)
    V2 = (xgoal[5:6]-x0[5:6])/((N-1)*dt)
    for k = 1:N-1
        xref[k+1][3] = x0[3]
        xref[k+1][4] = x0[4]
        xref[k+1][7] = x0[7]
        xref[k+1][8] = x0[8]
        xref[k+1][1:2] .= (xref[k][1:2]) + dt*V1
        xref[k+1][5:6] .= (xref[k][5:6]) + dt*V2
    end

    return SVector{8}.(xref)
end

nominal_trajectory (generic function with 1 method)

In [5]:
x0 = [   2.5; 0.0; pi/2; 0.0; 
        0.0; 2.5; 0.0; 0.0]

xgoal = [2.5; 5.0; pi/2; 0.0;
        5.0; 2.5; 0.0; 0.0]   
xref = nominal_trajectory(x0, xgoal, N, dt);

In [6]:
umax = [0.3, 0.3, 
        0.3, 0.3]   

# umin = [-2.0, -2.0, 
#         -2.0, -2.0, 
#         -2.0, -2.0]
umin = [-2.0, -2.0, 
        -2.0, -2.0]

ugoal = [   0.0, 0.0, 
            0.0, 0.0]  
            
solver.P .= rand(NHor, Nu, Nx)*0.01
solver.α = rand(NHor, Nu)*0.01

game.x0 .= x0
game.xref .= xref
game.xf .= xgoal
game.umin .= umin
game.umax .= umax
game.uf .= ugoal;

In [7]:
mutable struct BotPosition
    x::Float32
end

In [8]:
mutable struct BotsState
    x::Vector{Float32}
end

In [9]:
botsSt0 = BotsState(x0[1:4])
tbot0State = botsSt0.x
botsSt1 = BotsState(x0[5:8])
tbot1State = botsSt1.x

4-element Vector{Float32}:
 0.0
 2.5
 0.0
 0.0

In [10]:
function tb0_callback(msg)
    # x and y position of the robot
    xpos = msg.pose.pose.position.x
    ypos = msg.pose.pose.position.y

    # robot velocity
    vel = msg.twist.twist.linear.x

    # Robot pose
    qx = msg.pose.pose.orientation.x
    qy = msg.pose.pose.orientation.y
    qz = msg.pose.pose.orientation.z
    qw = msg.pose.pose.orientation.w

    theta = atan(2.0 * (qw * qz + qx * qy), 1.0 - 2.0 * (qy * qy + qz * qz))
    tbot0State .= [xpos; ypos; theta; vel]
end

function tb1_callback(msg)
    xpos = msg.pose.pose.position.x
    ypos = msg.pose.pose.position.y
    vel = msg.twist.twist.linear.x

    qx = msg.pose.pose.orientation.x
    qy = msg.pose.pose.orientation.y
    qz = msg.pose.pose.orientation.z
    qw = msg.pose.pose.orientation.w

    theta = atan(2.0 * (qw * qz + qx * qy), 1.0 - 2.0 * (qy * qy + qz * qz))
    tbot1State .= [xpos; ypos; theta; vel]
end

function commands(pub0, pub1)
    agent1 = Twist()
    agent2 = Twist()

    solver.P = rand(NHor, Nu, Nx)*0.01
    solver.α = rand(NHor, Nu)*0.01
    a = 0

    X = zeros(N, Nx) 
    U = zeros(N-1, Nu)
    X[1,:] = game.x0
    println("Game Setup done!")
    rate = Rate(10) # 10 Hz
    NTraj = N
    tstart = time_ns()
    for k = 1:N-1-NHor
        #println(tbot0State)
        #println("type: ", typeof([botsSt0.x; botsSt1.x]))
        #println("type: ", X[k])
        X[k,:] .= [tbot0State; tbot1State]
        #println(X[k,6])
        xₜ, uₜ = iLQGameSolver.solveILQGame(game, solver, iLQGameSolver.diffDrive4D, iLQGameSolver.costPointMass, X[k,:], k, false, Int(NTraj))
        X[k+1,:], U[k,:] = xₜ[2,:,:], uₜ[1,:,:]
        # println("State: ", X[k,:])
        # println("Input: ", U[k,:])
        agent1.linear.x = X[k+1,4]#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = U[k,2]#uₜ[i,2]



        agent2.linear.x = X[k+1,8]#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = U[k,4]#uₜ[i,4]

        NTraj = NTraj - 1

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
    end
    println(N)
    for k = N-NHor:N-1
        X[k,:] .= [tbot0State; tbot1State]
        xₜ, uₜ = iLQGameSolver.solveILQGame(game, solver, iLQGameSolver.diffDrive4D, iLQGameSolver.costPointMass, X[k,:], k, true, Int(NTraj))
        X[k+1,:], U[k,:] = xₜ[2,:,:], uₜ[1,:,:]

        agent1.linear.x = X[k+1,4]#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = U[k,2]#uₜ[i,2]


        agent2.linear.x = X[k+1,8]#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = U[k,4]#uₜ[i,4]

        NTraj = NTraj - 1

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
        #println(k)
    end
    
    tend = time_ns()
    rossleep(rate)

    for k = 1:10
        X[k,:] .= [tbot0State; tbot1State]
        #println("Done")
        agent1.linear.x = 0.0#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = 0.0#uₜ[i,2]


        agent2.linear.x = 0.0#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = 0.0#uₜ[i,4]

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
    end

    rate = N / (tend - tstart) * 1e9
    println("Controller ran at $rate Hz")
    
    return X;
    
end

function main()
    init_node("turtlebot3_cmd") # node name
    pub0 = Publisher{Twist}("/tb3_0/cmd_vel", queue_size=1) # topic name
    pub1 = Publisher{Twist}("/tb3_1/cmd_vel", queue_size=1) # topic name

    tbb1_odom = Subscriber{Odometry}("/tb3_0/odom", tb0_callback; queue_size=2)
    tbb2_odom = Subscriber{Odometry}("/tb3_1/odom", tb1_callback; queue_size=2)

    X = commands(pub0, pub1)

    #spin()
    #return X
end

if !isinteractive()
    main()
end


In [11]:
main()

Game Setup done!
600
Controller ran at 8.392822557888797 Hz


600×8 Matrix{Float64}:
 2.51211  4.94476     1.55264  …  2.48749  -0.0780049   0.0580471
 2.51224  4.95212     1.55297     2.48709  -0.0779271   0.0519506
 2.5123   4.9552      1.5526      2.48669  -0.0778708   0.0520858
 2.5123   4.9551      1.55267     2.48666  -0.0786663  -0.0413015
 2.51229  4.95505     1.55271     2.48674  -0.0782527  -0.000307086
 2.51229  4.95501     1.55276  …  2.48674  -0.0783556   3.64667e-6
 2.51229  4.95496     1.5528      2.48674  -0.0784636   3.64108e-6
 2.51229  4.95491     1.55284     2.48674  -0.0786801   3.63541e-6
 2.51228  4.95487     1.55288     2.48675  -0.0787886   3.63445e-6
 2.51228  4.95482     1.55293     2.48675  -0.0787886   3.63445e-6
 2.49917  0.00687965  1.58094  …  2.50085   0.0106693   0.0199463
 2.49915  0.00887076  1.58082     2.50085   0.0106693   0.0199463
 2.49912  0.0116524   1.58072     2.50088   0.0105594   0.028254
 ⋮                             ⋱  ⋮                    
 2.51053  4.84979     1.55348     2.49291  -0.0746928   0

In [ ]:
X

In [ ]:
testX = BotsState(x0[1:3])
tbotXtest = testX.x

In [ ]:
tbotXtest

In [ ]:
function changeX()
    tbotXtest .= [1; 2; 3]
end

function showX()
    println(tbotXtest)
end

In [ ]:
changeX()
showX()

In [ ]:
tbotXtest